In [1]:
import Pkg
using CSV
using DataFrames
using Base.Threads
using Statistics

In [2]:
# Paso 1: Leer el archivo CSV
df_movies = CSV.read("C:\\Users\\PC\\Downloads\\ml-25m\\ml-25m\\movies.csv", DataFrame)
df_ratings = CSV.read("C:\\Users\\PC\\Downloads\\ml-25m\\ml-25m\\ratings.csv", DataFrame)

Row,userId,movieId,rating,timestamp
,Int64,Int64,Float64,Int64
1,1,296,5.0,1147880044
2,1,306,3.5,1147868817
3,1,307,5.0,1147868828
4,1,665,5.0,1147878820
5,1,899,3.5,1147868510
6,1,1088,4.0,1147868495
7,1,1175,3.5,1147868826
8,1,1217,3.5,1147878326
9,1,1237,5.0,1147868839


In [19]:
# Paso 2: Dividir el DataFrame en 8 partes
function dividir_raitings_en_partes_iguales(num_partes, df_ratings)
    partes = [df_ratings[(i-1)*div(end, num_partes)+1:min(i*div(end, num_partes), end), :] for i in 1:num_partes-1]
    push!(partes, df_ratings[(num_partes-1)*div(size(df_ratings,1),num_partes):size(df_ratings,1),: ])
    return partes
end
#dividir_raitings_en_partes_iguales(6, df_ratings)

dividir_raitings_en_partes_iguales (generic function with 1 method)

In [4]:
# Paso 3: Separar el archivo movies
function movies_exploded_function(df_movies)
    df_movies.genre_list = map(x -> split(x, "|"), df_movies.genres)
    convert(Vector{Vector{String}}, df_movies.genre_list)
    lista_genero = []
    lista_movieId = []
    for i  in 1:size(df_movies,1)
        for j in 1:size(df_movies.genre_list[i:i][1],1)
            push!(lista_genero, df_movies.genre_list[i:i][1][j])
            push!(lista_movieId, df_movies.movieId[i:i][1])
        end
    end
    movies_exploded = DataFrame(movieId = lista_movieId, genre = lista_genero)
    return movies_exploded
end
#movies_exploded = movies_exploded_function(df_movies)

movies_exploded_function (generic function with 1 method)

In [13]:
# Paso 4: Crear funciones que haran el proceso

function ratings_per_genre_function(parte, movies_exploded)
    movies_ratings = innerjoin(movies_exploded, parte, on=:movieId)
    ratings_per_genre = combine(groupby(movies_ratings, :genre), nrow => :rating_count, :rating => mean)
    return ratings_per_genre
end

# Función de procesamiento
function procesar(parte, movies_exploded)
    ratings_per_genre_function(parte, movies_exploded)
end



procesar (generic function with 2 methods)

In [15]:
# Paso 5: Procesar en multihilos
function multihilos(num_partes, partes, movies_exploded)
    resultados = Vector{Any}(undef, num_partes)
    Threads.@threads for i in 1:num_partes
        resultados[i] = procesar(partes[i], movies_exploded)
    end
    return resultados
end
#resultados = multihilos(num_partes, partes)
# Mostrar resultados
#println(resultados)

multihilos (generic function with 2 methods)

In [24]:
# Paso 6: Combinar las 8 partes procesadas
function combinar_partes_procesadas(resultados, num_partes)
    combined_df = resultados[1]
    for i in 2:num_partes
        combined_df = vcat(combined_df, resultados[i])
    end
    combined_df = combine(groupby(combined_df, :genre), :rating_count => sum, :rating_mean => mean)
    combined_df = sort(combined_df, :genre)
    return combined_df
end

combinar_partes_procesadas (generic function with 2 methods)

In [47]:
combinar_partes_procesadas(resultados)

Row,genre,rating_count_sum,rating_mean_mean
,Any,Int64,Float64
1,(no genres listed),26627,3.3289
2,Action,7446917,3.46659
3,Adventure,5832423,3.51744
4,Animation,1630984,3.61495
5,Children,2124254,3.43251
6,Comedy,8926227,3.424
7,Crime,4190258,3.68504
8,Documentary,322449,3.70506
9,Drama,10962830,3.67719


In [25]:
function movie_lens(num_partes, df_movies, df_ratings)
    t0 = time()
    partes = dividir_raitings_en_partes_iguales(num_partes, df_ratings)
    movies_exploded = movies_exploded_function(df_movies)
    resultados = multihilos(num_partes, partes, movies_exploded)
    combined_df = combinar_partes_procesadas(resultados, num_partes)
    t1 = time()
    println("tiempo de ejecución: ", t1-t0)
    return combined_df
end

movie_lens (generic function with 1 method)

In [26]:
movie_lens(6, df_movies, df_ratings)

tiempo de ejecución: 16.07100009918213


Row,genre,rating_count_sum,rating_mean_mean
,Any,Int64,Float64
1,(no genres listed),26627,3.32945
2,Action,7446918,3.46658
3,Adventure,5832424,3.51743
4,Animation,1630987,3.61494
5,Children,2124258,3.43249
6,Comedy,8926230,3.424
7,Crime,4190260,3.68504
8,Documentary,322449,3.7054
9,Drama,10962834,3.67719
